In [69]:
# Charles Pedro
# Final Python and web project
# See accompanying web notes for more information about this script

import os
import wbdata
import pandas
import datetime

In [70]:
# these are some categories of indicators we can get from the World Bank
wbdata.get_source()

  id  name
----  --------------------------------------------------------------------
   1  Doing Business
   2  World Development Indicators
   3  Worldwide Governance Indicators
   5  Subnational Malnutrition Database
   6  International Debt Statistics
  11  Africa Development Indicators
  12  Education Statistics
  13  Enterprise Surveys
  14  Gender Statistics
  15  Global Economic Monitor
  16  Health Nutrition and Population Statistics
  18  IDA Results Measurement System
  19  Millennium Development Goals
  20  Quarterly Public Sector Debt
  22  Quarterly External Debt Statistics SDDS
  23  Quarterly External Debt Statistics GDDS
  24  Poverty and Equity
  25  Jobs
  27  Global Economic Prospects
  28  Global Financial Inclusion
  29  The Atlas of Social Protection: Indicators of Resilience and Equity
  30  Exporter Dynamics Database – Indicators at Country-Year Level
  31  Country Policy and Institutional Assessment
  32  Global Financial Development
  33  G20 Financial Inclus

In [71]:
# get list of countries that are low income or lower middle income
wbdata.get_country(incomelevel=('LIC', 'LMC'))

# save the list
country_list = [i['id'] for i in wbdata.get_country(incomelevel=('LIC', 'LMC'))]

# how many countries in those categories? 79
len(country_list)

79

In [72]:
# specify a date range of 10 years
date_range = datetime.datetime(2001, 1, 1), datetime.datetime(2010, 12, 31)

In [73]:
#### specify development indicators we want and label them
# official development assistance (ODA)
# GDP per capita growth
# percent of population with sanitation
# mortality rate
indicators = {"DT.ODA.ALLD.CD": "ODA", 
              "NY.GDP.PCAP.KD.ZG": "gdp_per_cap", 
              "SH.STA.BASS.ZS": "sant",
              "SH.DYN.MORT": "mort"}

In [74]:
# save to a dataframe
df = wbdata.get_dataframe(indicators, country=country_list, data_date=date_range)   

In [75]:
# let's see what we got
# it's a heirarchichal dataframe with two indexes - country and date
# we want to aggregate at the country level and get some metrics
df.head(20)

ODA  gdp_per_cap       sant   mort
country     date                                             
Afghanistan 2010  6.235320e+09    11.264133  34.177028   87.6
            2009  6.113120e+09    18.515369  32.909678   91.4
            2008  4.811210e+09     1.594211  31.654664   95.4
            2007  4.982610e+09    11.022774  30.411774   99.5
            2006  2.895830e+09     2.253357  29.180795  103.7
            2005  2.815100e+09     7.207933  28.038577  107.9
            2004  2.268530e+09    -2.875184  26.899573  112.1
            2003  1.538230e+09     3.868362  25.768887  116.3
            2002  1.223290e+09          NaN  24.645148  120.4
            2001  4.103600e+08          NaN  23.522815  124.6
Angola      2010  2.351700e+08     1.079148  41.060793  120.3
            2009  2.389400e+08    -2.808634  39.794270  128.6
            2008  3.690400e+08     7.116873  38.527510  137.4
            2007  2.599800e+08     9.890012  37.259303  146.8
            2006  1.635200e+08     7.582329  35.991282  156.2
            2005  4.146800e+08    11.030836  34.722637  165.6
            2004  1.144570e+09     7.187036  33.290392  174.6
            2003  4.935000e+08    -0.431851  31.856059  183.2
            2002  4.140300e+08     9.943764  30.422469  190.9
            2001  2.827300e+08     0.822114  28.989220  197.8

In [76]:
df.info()

<class 'wbdata.api.WBDataFrame'>
MultiIndex: 790 entries, ('Afghanistan', '2010') to ('Zimbabwe', '2001')
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ODA          780 non-null    float64
 1   gdp_per_cap  746 non-null    float64
 2   sant         778 non-null    float64
 3   mort         790 non-null    float64
dtypes: float64(4)
memory usage: 69.8+ KB


In [77]:
df.describe()

,ODA,gdp_per_cap,sant,mort
count,7.800000e+02,746.000000,778.000000,790.000000
mean,6.576543e+08,2.691667,41.511309,84.328608
std,8.754559e+08,4.537442,26.844218,46.460283
min,1.107000e+07,-31.333097,3.640189,11.600000
25%,1.699500e+08,0.614528,19.703025,43.250000
50%,3.730300e+08,2.828244,34.126380,81.900000
75%,8.083825e+08,5.034598,59.611325,114.800000
max,1.143196e+10,28.675962,98.517913,222.200000


In [78]:
# I will keep it simple for now and just do a sum and mean for the columns
df_aggregate = df.groupby(level=0)['ODA', 'gdp_per_cap', 'sant', 'mort'].agg({'ODA':'sum', 'gdp_per_cap':'mean'
        , 'sant':'mean', 'mort':'mean'}).copy()

<ipython-input-78-3dc58a681067>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_aggregate = df.groupby(level=0)['ODA', 'gdp_per_cap', 'sant', 'mort'].agg({'ODA':'sum', 'gdp_per_cap':'mean'


In [79]:
df_aggregate.head()

,ODA,gdp_per_cap,sant,mort
country,,,,
Afghanistan,3.329360e+10,6.606369,28.720894,105.89
Algeria,2.752730e+09,2.350878,85.394331,33.01
Angola,4.016160e+09,5.141163,35.191394,160.14
Bangladesh,1.221282e+10,4.060885,33.095282,63.81
Benin,4.423970e+09,0.886974,11.107469,122.16


In [80]:
# who got the most aid in that ten year period, in billions of US dollars?
df_aggregate['ODA'].sort_values(ascending=False).head(10).apply(lambda x: x/1000000000)

country
Afghanistan           33.29360
Nigeria               26.13365
Ethiopia              23.04519
Congo, Dem. Rep.      21.79920
Tanzania              20.65786
Vietnam               19.66074
Pakistan              18.17368
India                 16.85531
Mozambique            16.13926
West Bank and Gaza    16.07526
Name: ODA, dtype: float64

In [81]:
# who had the highest average GDP per capita growth in that period? 
df_aggregate['gdp_per_cap'].sort_values(ascending=False).head(10)

country
Myanmar        11.094621
Bhutan          7.330670
Chad            7.141927
Afghanistan     6.606369
Cambodia        6.235245
Tajikistan      6.102577
Ethiopia        5.744553
Rwanda          5.709825
Vietnam         5.589638
Moldova         5.437783
Name: gdp_per_cap, dtype: float64

In [82]:
# Here I calculate a metric to find which countries are best using development aid to improve GDP, sanitation, 
# and mortality rate
df_aggregate['metric'] = (-pow(df_aggregate['ODA']/1000000000, 0.5) + df_aggregate['gdp_per_cap']/4 + 
    df_aggregate['sant']*.5 - df_aggregate['mort']*0.5 + 80)*0.2

df_aggregate.head(10)

,ODA,gdp_per_cap,sant,mort,metric
country,,,,,
Afghanistan,3.329360e+10,6.606369,28.720894,105.89,7.459396
Algeria,2.752730e+09,2.350878,85.394331,33.01,21.024150
Angola,4.016160e+09,5.141163,35.191394,160.14,3.361390
Bangladesh,1.221282e+10,4.060885,33.095282,63.81,12.432636
Benin,4.423970e+09,0.886974,11.107469,122.16,4.518431
Bhutan,8.040800e+08,7.330670,56.275740,56.41,16.173766
Bolivia,6.295970e+09,2.025441,41.045483,55.10,14.193985
Burkina Faso,7.689590e+09,2.859118,14.542883,149.30,2.112642
Burundi,3.923520e+09,0.235970,45.825971,120.43,8.155238


In [83]:
# what are the top 10 countries based on this metric? 
df_aggregate['metric'].sort_values(ascending=False).head(10)

country
Ukraine                 23.884365
Syrian Arab Republic    23.196836
Sri Lanka               23.039087
West Bank and Gaza      21.863446
Tunisia                 21.757889
El Salvador             21.745291
Moldova                 21.434708
Kyrgyz Republic         21.398144
Egypt, Arab Rep.        21.202374
Algeria                 21.024150
Name: metric, dtype: float64

In [84]:
# what are the bottom 10 countries based on this metric? 
df_aggregate['metric'].sort_values().head(10)

country
Sierra Leone               -2.348065
Niger                      -0.232477
Chad                        0.517097
Central African Republic    1.770172
Burkina Faso                2.112642
Mali                        2.424882
Nigeria                     2.956624
Guinea-Bissau               3.210827
Angola                      3.361390
Guinea                      3.626688
Name: metric, dtype: float64